In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

## 1. Создайте RDD из файла на HDFS

In [1]:
file_rdd = sc.textFile("/user/atitov/data1.json")

In [2]:
file_rdd.take(2)

[u'{ "name":"Moscow", "country":"Russia", "continent": "Europe", "population": 12380664}',
 u'{ "name":"Madrid", "country":"Spain" }']

In [3]:
import json

## 2. Получите список всех ключей

In [4]:
def get_json(x):
    try:
        return json.loads(x)
    except:
        return {}

In [5]:
distinct_keys = file_rdd.flatMap(lambda x: get_json(x).keys()).distinct().collect()

In [6]:
distinct_keys

[u'country', u'population', u'continent', u'name']

## 3. Исключите дубликаты из RDD

In [7]:
unique = file_rdd.map(lambda x: get_json(x))\
                 .map(lambda x: ((x.get('country'), x.get('name')), x))\
                 #???????????????
                 .reduceByKey(lambda x,y: x)\
                 .map(lambda x: x[1])

unique.collect()

[{u'continent': u'Europe',
  u'country': u'Germany',
  u'name': u'Berlin',
  u'population': 3490105},
 {},
 {u'continent': u'Europe', u'country': u'Spain', u'name': u'Barselona'},
 {u'continent': u'Africa',
  u'country': u'Egypt',
  u'name': u'Cairo',
  u'population': 11922948},
 {u'continent': u'Europe',
  u'country': u'Russia',
  u'name': u'Moscow',
  u'population': 12380664},
 {u'country': u'Spain', u'name': u'Madrid'},
 {u'continent': u'Europe',
  u'country': u'France',
  u'name': u'Paris',
  u'population': 2196936}]

In [8]:
#???????????????
unique = unique.filter(lambda x: x)

unique.collect()

[{u'continent': u'Europe',
  u'country': u'Germany',
  u'name': u'Berlin',
  u'population': 3490105},
 {u'continent': u'Europe', u'country': u'Spain', u'name': u'Barselona'},
 {u'continent': u'Africa',
  u'country': u'Egypt',
  u'name': u'Cairo',
  u'population': 11922948},
 {u'continent': u'Europe',
  u'country': u'Russia',
  u'name': u'Moscow',
  u'population': 12380664},
 {u'country': u'Spain', u'name': u'Madrid'},
 {u'continent': u'Europe',
  u'country': u'France',
  u'name': u'Paris',
  u'population': 2196936}]

## 5. Найдите город с самым большим населением

In [9]:
unique.max(lambda x: x.get('population'))['name']

u'Moscow'

## 6. Посчитайте топ-2 континентов по населению

In [10]:
top2_continents = unique.map(lambda x: (x.get('continent', 'Earth'), x.get('population', 0)))\
                        .reduceByKey(lambda x, y: x + y)\
                        .sortByKey(ascending=False, keyfunc=lambda x: x[1]).collect()[:2]

## 7. Добавьте к каждой записи новое поле

In [11]:
#???????????????
populous_bc = sc.broadcast(top2_continents[0][0])

In [12]:
top2_continents[0][0]

u'Europe'

In [56]:
def add_key(dct, key, value):
    dct[key] = value
    return dct

In [57]:
top2_continents

[(u'Europe', 18067705), (u'Africa', 11922948)]

In [64]:
rdd_with_new_filed = unique\
            .map(lambda x: add_key(x, 'is_popular', x.get('continent', 'Earth') == populous_bc.value))

In [65]:
rdd_with_new_filed.take(10)

[{u'continent': u'Europe',
  u'country': u'Germany',
  'is_popular': True,
  u'name': u'Berlin',
  u'population': 3490105},
 {u'continent': u'Europe',
  u'country': u'Spain',
  'is_popular': True,
  u'name': u'Barselona'},
 {u'continent': u'Africa',
  u'country': u'Egypt',
  'is_popular': False,
  u'name': u'Cairo',
  u'population': 11922948},
 {u'continent': u'Europe',
  u'country': u'Russia',
  'is_popular': True,
  u'name': u'Moscow',
  u'population': 12380664},
 {u'country': u'Spain', 'is_popular': False, u'name': u'Madrid'},
 {u'continent': u'Europe',
  u'country': u'France',
  'is_popular': True,
  u'name': u'Paris',
  u'population': 2196936}]

## 8. Измените партиционирование RDD по значению поля continent

In [66]:
def new_partitioner(key):
    return hash(key)

rdd_with_new_filed\
    .map(lambda x: (x.get('continent', 'Earth'), x))\
    .repartitionAndSortWithinPartitions(5, new_partitioner, True)\
    .map(lambda x: x[1])\
    .glom()\
    .collect()

[[],
 [],
 [{u'continent': u'Europe',
   u'country': u'Germany',
   'is_popular': True,
   u'name': u'Berlin',
   u'population': 3490105},
  {u'continent': u'Europe',
   u'country': u'Spain',
   'is_popular': True,
   u'name': u'Barselona'},
  {u'continent': u'Europe',
   u'country': u'Russia',
   'is_popular': True,
   u'name': u'Moscow',
   u'population': 12380664},
  {u'continent': u'Europe',
   u'country': u'France',
   'is_popular': True,
   u'name': u'Paris',
   u'population': 2196936}],
 [{u'country': u'Spain', 'is_popular': False, u'name': u'Madrid'}],
 [{u'continent': u'Africa',
   u'country': u'Egypt',
   'is_popular': False,
   u'name': u'Cairo',
   u'population': 11922948}]]